# **Coursera capstone project week 3 - part 1 and 2**

####  by Francesca Scalesse

In this project we explore, segment, and cluster the neighborhoods in the city of Toronto based on the postalcode and borough information. Th project will be divided in 3 part and every one will contains the previous ones. 

In this notebook it will be shown the first and second part of the final assigment for the week 3 of the coursera course: *Applied data science capstone*.

## Part 1

First of all data about Toronto neighborhood will be scraped using BeautifulSoup. Then the acquired data will be transformed in a dataframe using pandas.

### Import needed libraries 

In [39]:
#!pip install pandas
#!pip install requests
!pip install bs4
#!pip install plotly

In [40]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

### Create a beautiful soup object

In the following section the data about the city of Toronto will be retrieved by the URL and a beautiful soup object will be created. The source of the data is the wikipedia webpage about Toronto Neighborhood data (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)

In [41]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [42]:
html_data = requests.get(url).text

In [43]:
soup = BeautifulSoup(html_data, "html5lib")

Now the beautiful soup object will be used to create a dataframe using pandas. The dataframe (df) will have 3 columns: Postal_code, Borough and Neighborhood. If cells has a Borough value "Not assigned" they will be ignored.If a cell has a borough but a "Not assigned"  neighborhood, then the neighborhood will be the same as the borough.

In [44]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)


df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})



Now the first 5 rows of the dataframe will be shown

In [45]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [46]:
df.shape

(103, 3)

This data frame has 103 rows and 3 columns

## Part 2

In this section the dataframe will be updated adding the coordinate columns. In this way it will be possible explore Toronto's neighborhood using foursquare in the third part of the project. The data will be imported from their url and transformed in a dataframe. The neede libraries are already imported.

### Import data and transform in a dataframe

The name of the first column 

In [49]:
url_geospatial = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv"
df2 = pd.read_csv(url_geospatial)
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [51]:
df2.shape

(103, 3)

This data frame has 103 rows and 3 columns

In [56]:
print(list(df2)[0])
print(list(df)[0])

Postal Code
PostalCode


### Merge the dataframes

Now we need to merge our two dataframes in a new one named "df3". Both of them have the same number of rows so we can easily use the method merge(). However we need first change the name of thi first column of dataframe "df2". In this way it will be possible to merge the dataframes using the column 'PostalCode'

**Rename column**

In [57]:
df2.rename(columns = {'Postal Code':'PostalCode'},inplace=True)
df2.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**Merge the dataframes**

In [60]:
df3 = pd.merge(left = df, right = df2, how = 'outer', on = 'PostalCode' )
df3.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
